In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import datetime
import sys
import tempfile
from dateutil import tz

from ratschlab_common.io.bigmatrix import TablesBigMatrixReader, TablesBigMatrixWriter

# Big Matrix Usage Example

`Bigmatrix` represents an abstraction for out-of-core computation with large
matrices having some 'metadata' associated with the rows and columns. This
metadata can be for example some patient id and a timestamp for the rows
and a column name for each column, e.g. the name of features. The matrix itself
could represent the values of the features for patients at different times.

The motivation behind this abstraction is, that sometimes a dataset can be almost
represented as a matrix, i.e. has the same datatype across all columns, typically
some float or int. But a few columns represent something else, e.g. time information
or ids. The idea is to handle these metadata separately from the rest. This way
the entire dataset can be handled more efficiently and easily.

`ratschlab_common.io.bigmatrix` serializes all the data to disk using HDF5 via the `pytables` library. A file generated by this module consists of three nodes: one for the matrix, one for the column description and one for the row description. The matrix is written directly by `pytables` whereas the metadata is written by `pandas`. This can be changed to `pytables` as well, for example in case `pandas` would use too much memory for processing the metadata.  

## Generating Fake Data

First, we'll generate some fake data and write it incrementally using `TablesBigMatrixWriter`.

The fake data consists of numerical records of several patients, where each record has a timestamp and a patient id associated. These two columns are stored as row descriptions (i.e. a dataframe). 

Note, that in principle, the time information could be integrated into the numerical matrix. However, indexing and time zone handling etc would have to be done manually. In this approach, we let `pandas` take care of this.

In [2]:
seed = 42**2 # make it deterministic (at least when running from top to bottom)

np.random.seed(seed)

## config:
num_patients = 50
patient_ids = np.random.randint(100000, size=num_patients)

rows_per_patient = 1000
cols = 150

start_date = datetime.datetime(2018, 3, 1)

In [3]:
data_size_in_memory = 8*num_patients*rows_per_patient*cols/1024**3 #GB
data_size_in_memory

0.05587935447692871

In [4]:
def fake_patient_data(pid, rows, cols):
    """
    Generates fake data for one patient with a given patient id
    """
    ids = np.full((rows,), pid)
    
    # a measurement all 5 minutes
    timestamps = pd.date_range(pd.to_datetime(start_date), periods=rows, freq='5min')
    
    row_desc = pd.DataFrame.from_dict({'id' : ids, 'ts' : timestamps})
    
    data = np.random.uniform(size=(rows, cols)) + pid # biasing the values, s.t. we know we got the data from the right patient
    
    return data, row_desc

In [5]:
temp_dir = tempfile.TemporaryDirectory()

output_path = Path(temp_dir.name, 'fake_patient_data.h5')
output_path

PosixPath('/var/folders/ry/d9fpyr5d21xgbjyrl084ssqh0000gn/T/tmp1gs7pni_/fake_patient_data.h5')

Also columns can be described using a data frame. Typically, it would contain just the column name. Here we add also some comment.

In [6]:
col_desc = pd.DataFrame(
    {'name': ["my_col{}".format(c) for c in range(0, cols)],
     'comment': ["You could additionally also add some comment for column {}".format(c) for c in range(0, cols)]
    })

col_desc.head()

,comment,name
0,You could additionally also add some comment f...,my_col0
1,You could additionally also add some comment f...,my_col1
2,You could additionally also add some comment f...,my_col2
3,You could additionally also add some comment f...,my_col3
4,You could additionally also add some comment f...,my_col4


In order to allow efficient access and compression when writing to disk using HDF5, data can be split into "chunks".
See also https://support.hdfgroup.org/HDF5/doc/Advanced/Chunking/index.html

Poorly chosen chunksizes/shapes can have a significantly negative impact on performance, so it's usually worth experimenting a bit.

In general, when writing HDF5 files incrementally, it is important to select a chunksize/shape manually, as otherwise the HDF5 library (e.g. pytables) may choose an inappropriate size depending on the first data written.

For some advice on how to choose the chunksize, see http://www.pytables.org/usersguide/optimization.html#understanding-chunking
Since blosc compression is currently used as default one should aim at something in the ballpark of 500kb of data per chunk.


In [7]:
chunkshape = (1000, 50) # nicely aligned with the fake data (doesn't need to be the case)

chunksize_kb = np.dtype('float').itemsize * chunkshape[0] * chunkshape[1]/1024
chunksize_kb

390.625

Since we assume we are dealing with a lot of data, we are going to write the patient data sequentially.

Note, that only rows can be appended, but not columns.

In [8]:
writer = TablesBigMatrixWriter()

if output_path.exists():
    output_path.unlink()
    
for pid in patient_ids:
    data, row_desc = fake_patient_data(pid, rows_per_patient, cols)
    
    writer.write_or_append(output_path, data, row_desc, col_desc, chunkshape=chunkshape)
    

In [9]:
!ls -l $output_path

-rw-r--r--  1 marc  staff  38409242 May 16 10:43 /var/folders/ry/d9fpyr5d21xgbjyrl084ssqh0000gn/T/tmp1gs7pni_/fake_patient_data.h5


Below the groups in the newly created HDF5 file

In [10]:
!ptdump $output_path

/ (RootGroup) ''
/data (EArray(50000, 150), fletcher32, shuffle, blosc:lz4(5)) ''
/col_descr (Group) ''
/col_descr/table (Table(150,), shuffle, blosc:lz4(5)) ''
/row_descr (Group) ''
/row_descr/table (Table(50000,), shuffle, blosc:lz4(5)) ''


## Reading Back Data

In [11]:
# reading back data of a "random" patient
some_id = patient_ids[-1]
some_id

26710

In [12]:
# getting all the data of some patient
with TablesBigMatrixReader(output_path) as reader:
    # first getting the row indices of the data belonging to the given patient
    row_indices = reader.get_row_indices('id == {}'.format(some_id))
    
    # accessing the matrix
    data_tmp = reader[row_indices, :]
data_tmp

array([[26710.20143677, 26710.55182748, 26710.87855975, ...,
        26710.81713081, 26710.73815005, 26710.11845951],
       [26710.78651242, 26710.69449985, 26710.29312988, ...,
        26710.6971596 , 26710.43853235, 26710.15546655],
       [26710.84996235, 26710.45633331, 26710.99427889, ...,
        26710.87577065, 26710.38067227, 26710.93668617],
       ...,
       [26710.3633043 , 26710.80679326, 26710.86311316, ...,
        26710.04492171, 26710.37460093, 26710.53402499],
       [26710.65226909, 26710.07424714, 26710.88747023, ...,
        26710.22762388, 26710.1339891 , 26710.48514644],
       [26710.02089059, 26710.13063079, 26710.80944825, ...,
        26710.5185127 , 26710.93133834, 26710.50079785]])

We can also get data on a specific day. The code uses pandas' `dataframe.query` function. Query syntax is a bit cumbersome...

Note that '2018-03-02' implicitly gets converted to '2018-03-02 00:00:00' local time!

In [13]:
with TablesBigMatrixReader(output_path) as reader:
    row_indices = reader.get_row_indices("id == {} & ts >= '2018-03-02' & ts < '2018-03-03'".format(some_id))
    
    row_desc = reader.row_desc()
    timestamps = row_desc.loc[row_indices, 'ts']
    
    data_tmp = reader[row_indices, :]
timestamps[1:10], data_tmp[1:10, ]

(49289   2018-03-02 00:05:00
 49290   2018-03-02 00:10:00
 49291   2018-03-02 00:15:00
 49292   2018-03-02 00:20:00
 49293   2018-03-02 00:25:00
 49294   2018-03-02 00:30:00
 49295   2018-03-02 00:35:00
 49296   2018-03-02 00:40:00
 49297   2018-03-02 00:45:00
 Name: ts, dtype: datetime64[ns],
 array([[26710.90660873, 26710.8418034 , 26710.71130172, ...,
         26710.28144534, 26710.50091314, 26710.55610162],
        [26710.72233082, 26710.00978424, 26710.93947081, ...,
         26710.28969285, 26710.95075967, 26710.31592936],
        [26710.22408737, 26710.19909297, 26710.86823822, ...,
         26710.44479405, 26710.02135051, 26710.70572147],
        ...,
        [26710.312206  , 26710.72083411, 26710.4207162 , ...,
         26710.16075147, 26710.0129967 , 26710.90915809],
        [26710.40858287, 26710.4680696 , 26710.54513204, ...,
         26710.82508201, 26710.99749598, 26710.82239922],
        [26710.41591162, 26710.45859962, 26710.34461571, ...,
         26710.86537393, 26710

In [14]:
# getting only a few columns of data for some patient
with TablesBigMatrixReader(output_path) as reader:
    row_indices = reader.get_row_indices('id == {}'.format(some_id))
    col_indices = reader.get_col_indices_by_name(['my_col5', 'my_col6'])
    
    data_tmp = reader[row_indices, col_indices]
data_tmp

array([[26710.29889554, 26710.66328001],
       [26710.38902207, 26710.82468611],
       [26710.91017141, 26710.85773244],
       ...,
       [26710.9989812 , 26710.22916032],
       [26710.85948349, 26710.57984862],
       [26710.30660865, 26710.45473952]])

In [15]:
# calculating the median over an entire column (i.e. over all patients)
with TablesBigMatrixReader(output_path) as reader:
    col_indices = reader.get_col_indices_by_name(['my_col7'])
    
    col_median = np.median(reader[:, col_indices])
col_median

40353.499702672576

In [16]:
temp_dir.cleanup()